In [32]:
import os, sys
import re
import nltk
import gensim
import pandas as pd
import xml.etree.ElementTree as ET
from collections import defaultdict
from gensim import corpora
sys.path.insert(0, '../scripts')
from WikiPage import WikiPage

In [2]:
os.chdir('../data')
xmlfile = 'Wikipedia-dog.xml'

In [3]:
tree = ET.parse(xmlfile)
root = tree.getroot()

In [4]:
test_index = 100
test = WikiPage(root[test_index])

In [5]:
test.data

,website,title,page_type,text_raw
17701542,https://en.wikipedia.org/wiki/KattenKabinet,KattenKabinet,article,[[File:Cat Cabinet face Amsterdam-Photo by Pej...


In [6]:
def clean_raw_text(text_raw, searchstr):
    m = re.search(searchstr, text_raw)
    if m:
        text_raw = re.sub(searchstr, ' ', text_raw)
        text_raw = clean_raw_text(text_raw, searchstr)
    return text_raw

In [7]:
print(test.text_raw[:1000])

[[File:Cat Cabinet face Amsterdam-Photo by Pejman Akbarzadeh Persian Dutch Network.jpg|thumb|The KattenKabinet in [[Amsterdam]]]]

The '''KattenKabinet''' ("Cat Cabinet") is an art museum in [[Amsterdam]] devoted to works depicting [[cat]]s. The museum collection includes paintings, drawings, sculptures and other works of art by [[Pablo Picasso]], [[Rembrandt]], [[Henri de Toulouse-Lautrec]], [[Guillaume Cornelis van Beverloo|Corneille]], [[Sal Meijer]], [[Théophile Steinlen]], and [[Jože Ciuha]], among others.

The museum is housed in a canalside building at [[Herengracht]] 497, in the grand ''[[Gouden Bocht]]'' ("Golden Bend") of this canal. The owner lives on the second floor of the building with his family. There are some cats in the museum as well.

== History == 
The house and the adjacent building at Herengracht 499 were built in 1667 for the patrician brothers Willem and Adriaen van Loon. Through the luck of the draw, Willem was given the house at number 497. Later, the house w

In [8]:
clean_raw_text(test.text_raw, '[\(\[].*?[\)\]]')

" ]]]\n\nThe '''KattenKabinet'''   is an art museum in  ] devoted to works depicting  ]s. The museum collection includes paintings, drawings, sculptures and other works of art by  ],  ],  ],  ],  ],  ], and  ], among others.\n\nThe museum is housed in a canalside building at  ] 497, in the grand '' ]''   of this canal. The owner lives on the second floor of the building with his family. There are some cats in the museum as well.\n\n== History == \nThe house and the adjacent building at Herengracht 499 were built in 1667 for the patrician brothers Willem and Adriaen van Loon. Through the luck of the draw, Willem was given the house at number 497. Later, the house was inhabited by Amsterdam mayor  ] and Amsterdam pensionary  ], among others.  ] visited Van Berckel at the house during his time as U.S. ambassador to the Dutch republic.\n\nIn 1985, the building was restored, and in 1990 the museum was founded by Bob Meijer in memory of his red tomcat John Pierpont Morgan  ]).\n\nThe museum 

In [9]:
text = re.sub(r'[^\w]', ' ', test.text_raw)
text

'  File Cat Cabinet face Amsterdam Photo by Pejman Akbarzadeh Persian Dutch Network jpg thumb The KattenKabinet in   Amsterdam      The    KattenKabinet      Cat Cabinet   is an art museum in   Amsterdam   devoted to works depicting   cat  s  The museum collection includes paintings  drawings  sculptures and other works of art by   Pablo Picasso      Rembrandt      Henri de Toulouse Lautrec      Guillaume Cornelis van Beverloo Corneille      Sal Meijer      Théophile Steinlen    and   Jože Ciuha    among others   The museum is housed in a canalside building at   Herengracht   497  in the grand     Gouden Bocht       Golden Bend   of this canal  The owner lives on the second floor of the building with his family  There are some cats in the museum as well      History     The house and the adjacent building at Herengracht 499 were built in 1667 for the patrician brothers Willem and Adriaen van Loon  Through the luck of the draw  Willem was given the house at number 497  Later  the house 

In [10]:
tokenizer = nltk.RegexpTokenizer(r'\w+')
ttext = tokenizer.tokenize(test.text_raw)
print(ttext)

['File', 'Cat', 'Cabinet', 'face', 'Amsterdam', 'Photo', 'by', 'Pejman', 'Akbarzadeh', 'Persian', 'Dutch', 'Network', 'jpg', 'thumb', 'The', 'KattenKabinet', 'in', 'Amsterdam', 'The', 'KattenKabinet', 'Cat', 'Cabinet', 'is', 'an', 'art', 'museum', 'in', 'Amsterdam', 'devoted', 'to', 'works', 'depicting', 'cat', 's', 'The', 'museum', 'collection', 'includes', 'paintings', 'drawings', 'sculptures', 'and', 'other', 'works', 'of', 'art', 'by', 'Pablo', 'Picasso', 'Rembrandt', 'Henri', 'de', 'Toulouse', 'Lautrec', 'Guillaume', 'Cornelis', 'van', 'Beverloo', 'Corneille', 'Sal', 'Meijer', 'Théophile', 'Steinlen', 'and', 'Jože', 'Ciuha', 'among', 'others', 'The', 'museum', 'is', 'housed', 'in', 'a', 'canalside', 'building', 'at', 'Herengracht', '497', 'in', 'the', 'grand', 'Gouden', 'Bocht', 'Golden', 'Bend', 'of', 'this', 'canal', 'The', 'owner', 'lives', 'on', 'the', 'second', 'floor', 'of', 'the', 'building', 'with', 'his', 'family', 'There', 'are', 'some', 'cats', 'in', 'the', 'museum', 'a

In [12]:
df = pd.read_csv('../data/Wikipedia-dog.csv', index_col=0)

In [13]:
df.head()

,website,title,page_type,text_raw
970284,https://en.wikipedia.org/wiki/Category:Dog_sho...,Category:Dog shows and showing,category,{{Cat main|Conformation show|Show dog}}\n{{por...
972913,https://en.wikipedia.org/wiki/Category:Dog_health,Category:Dog health,category,This is a collection of articles about the hea...
970251,https://en.wikipedia.org/wiki/Category:Dog_org...,Category:Dog organizations,category,This is an automatically collected list of art...
729436,https://en.wikipedia.org/wiki/Category:Dog_sports,Category:Dog sports,category,This is an automatically accumulated list of a...
978163,https://en.wikipedia.org/wiki/Category:Dogs_as...,Category:Dogs as pets,category,[[Category:Dogs|Pets]]\n[[Category:Mammals as ...


In [14]:
documents = list(df.text_raw.values)
documents[0]

'{{Cat main|Conformation show|Show dog}}\n{{portal|Dogs}}\nThis is an automatically collected list of articles related to showing dogs for their appearance in [[conformation show]]s.\n\n[[Category:Dogs|Shows and showing]]\n[[Category:Competitions]]\n[[Category:Animal shows| Dog]]'

In [15]:
#nltk.download()

In [18]:
stoplist = set(nltk.corpus.stopwords.words('english'))

In [25]:
texts = [[word for word in re.sub(r'[^\w]', ' ', document.lower()).split() \
          if word not in stoplist] \
         for document in documents]

In [26]:
frequency = defaultdict(int)

In [27]:
for text in texts:
     for token in text:
         frequency[token] += 1

In [28]:
texts = [[token for token in text if frequency[token] > 1]
          for text in texts]

In [37]:
dictionary = corpora.Dictionary(texts)
#dictionary.save('/tmp/wikipedia-dog.dict')  # store the dictionary, for future reference
print(dictionary)
print(dictionary.token2id)

Dictionary(12988 unique tokens: ['cat', 'main', 'conformation', 'show', 'dog']...)
{'cat': 0, 'main': 1, 'conformation': 2, 'show': 3, 'dog': 4, 'portal': 5, 'dogs': 6, 'this': 7, 'is': 8, 'an': 9, 'automatically': 10, 'collected': 11, 'list': 12, 'articles': 13, 'related': 14, 'showing': 15, 'their': 16, 'appearance': 17, 's': 18, 'category': 19, 'shows': 20, 'competitions': 21, 'animal': 22, 'collection': 23, 'about': 24, 'health': 25, 'care': 26, 'primarily': 27, 'domestic': 28, 'includes': 29, 'medical': 30, 'conditions': 31, 'issues': 32, 'parasites': 33, 'surgical': 34, 'procedures': 35, 'similar': 36, 'topics': 37, 'commons': 38, 'mammal': 39, 'organizations': 40, 'sports': 41, 'involving': 42, 'animals': 43, 'sport': 44, 'pets': 45, 'mammals': 46, 'as': 47, 'equipment': 48, 'pet': 49, 'collects': 50, 'information': 51, 'how': 52, 'breed': 53, 'why': 54, 'people': 55, 'canine': 56, 'reproduction': 57, 'breeding': 58, 'see': 59, 'also': 60, 'domestication': 61, 'artificial': 62, 

In [39]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('/tmp/deerwester.mm', corpus)
print(corpus)

[[(0, 1), (1, 1), (2, 2), (3, 3), (4, 2), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 3), (20, 2), (21, 1), (22, 1)], [(0, 2), (1, 1), (4, 2), (6, 3), (7, 2), (8, 1), (13, 1), (19, 2), (23, 1), (24, 1), (25, 6), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)], [(4, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (18, 1), (19, 2), (22, 1), (24, 1), (40, 3)], [(0, 1), (1, 1), (4, 2), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (12, 1), (13, 1), (19, 3), (24, 1), (38, 1), (41, 4), (42, 1), (43, 1), (44, 1)], [(6, 1), (19, 2), (45, 2), (46, 1), (47, 1)], [(0, 1), (4, 1), (6, 1), (19, 2), (38, 1), (48, 3), (49, 1)], [(0, 2), (1, 1), (4, 1), (6, 3), (7, 1), (19, 4), (22, 1), (24, 1), (38, 1), (46, 1), (50, 1), (51, 1), (52, 2), (53, 2), (54, 1), (55, 1), (56, 1), (57, 2), (58, 3), (59, 1), (60, 1), (61, 1

In [ ]:
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
        # assume there's one document per line, tokens separated by whitespace
        yield dictionary.doc2bow(line.lower().split())

In [ ]:
corpus_memory_friendly = MyCorpus()  # doesn't load the corpus into memory!
print(corpus_memory_friendly)
<__main__.MyCorpus object at 0x10d5690>